In [29]:
import xmlrpc.client 
import pandas as pd

# Conexión con Odoo
username = "juan.cano@donsson.com"  # tu usuario
password = "1000285668"         # tu contraseña
url = "https://donsson.com"     # URL del servidor
db = "Donsson_produccion" # nombre de la base de datos

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")




fields_info = models.execute_kw(
    db, uid, password,
    'sale.order',  # Nombre técnico del modelo
    'fields_get',
    []
)

# Convertir a DataFrame para visualizar mejor
df_campos = pd.DataFrame.from_dict(fields_info, orient='index')
df_campos

,sortable,help,searchable,required,manual,readonly,depends,groups,company_dependent,translate,...,domain,context,states,relation_field,selection,digits,size,change_default,related,sanitize
origin,True,Reference of the document that generated this ...,True,False,False,False,[],False,False,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
message_follower_ids,False,NaN,True,False,False,False,[],False,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
create_date,True,Date on which sales order is created.,True,False,False,True,[],False,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
store_id,True,Sucursal.,True,True,False,False,[],False,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
categ_ids,True,NaN,True,False,False,False,[],False,False,NaN,...,"['|', ('section_id', '=', section_id), ('secti...",{'object_name': 'crm.lead'},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
write_date,True,NaN,True,False,False,False,[],False,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
website_order_line,True,Order Lines to be displayed on the website. Th...,True,False,False,True,[],False,False,NaN,...,NaN,NaN,NaN,order_id,NaN,NaN,NaN,NaN,NaN,NaN
website_message_ids,True,Website communication history,True,False,False,False,[],False,False,NaN,...,"[&, [model, =, sale.order], [type, =, comment]]",NaN,NaN,res_id,NaN,NaN,NaN,NaN,NaN,NaN
product_id,False,NaN,True,False,False,False,[],False,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# TOTAL FACTURAS 1 AÑO

In [30]:
import xmlrpc.client 
from datetime import date
import pandas as pd

# Conexión con Odoo
username = "juan.cano@donsson.com"  # tu usuario
password = "1000285668"         # tu contraseña
url = "https://donsson.com"     # URL del servidor
db = "Donsson_produccion" # nombre de la base de datos


# --- Autenticación ---
common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")

# --- Fechas ---
from datetime import date, timedelta

# --- Fechas automáticas ---
hoy = date.today()
fecha_fin = hoy.strftime("%Y-%m-%d")
fecha_inicio = (hoy - timedelta(weeks=52)).strftime("%Y-%m-%d")


# --- 1) Buscar facturas válidas ---
invoice_domain = [
    ("date_invoice", ">=", fecha_inicio),
    ("date_invoice", "<=", fecha_fin),
    ("type", "=", "out_invoice"),   # solo ventas
    ("state", "in", ["open", "paid"])
]

invoice_ids = models.execute_kw(
    db, uid, password,
    "account.invoice", "search",
    [invoice_domain]
)
print(f"Facturas encontradas: {len(invoice_ids)}")

# --- 2) Descargar las líneas de esas facturas ---
fields = ["product_id", "quantity", "price_subtotal", "invoice_id","create_date"]

records = []
limit = 20000
offset = 0

while True:
    result = models.execute_kw(
        db, uid, password,
        "account.invoice.line", "search_read",
        [[("invoice_id", "in", invoice_ids)]],
        {"fields": fields, "limit": limit, "offset": offset}
    )
    if not result:
        break
    records.extend(result)
    offset += limit
    print(f"Descargados {len(records)} registros...")

# --- 3) Pasar a DataFrame ---
df = pd.DataFrame(records).fillna(0)

# --- 3) Pasar a DataFrame ---
df = pd.DataFrame(records).fillna(0)

# Separar product_id
df["product_id_num"] = df["product_id"].apply(
    lambda x: x[0] if isinstance(x, (list, tuple)) else None
)
df["product_name"] = df["product_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) else str(x)
)

# Separar invoice_id
df["invoice_id_num"] = df["invoice_id"].apply(
    lambda x: x[0] if isinstance(x, (list, tuple)) else None
)
df["invoice_name"] = df["invoice_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) else str(x)
)

# Convertir fecha a datetime
df["date_invoice"] = pd.to_datetime(df["create_date"], errors="coerce")

# Eliminar las columnas originales problemáticas
df = df.drop(columns=["product_id", "invoice_id","create_date"])


df.to_parquet("historico_ventas_1_año.parquet", index=False)
print(f"Total de líneas descargadas: {len(df)}")

Facturas encontradas: 50692


KeyboardInterrupt: 

# FACTURAS ULTIMAS 12 SEMANAS

In [ ]:
import xmlrpc.client 
from datetime import date
import pandas as pd

# Conexión con Odoo
username = "juan.cano@donsson.com"  # tu usuario
password = "1000285668"         # tu contraseña
url = "https://donsson.com"     # URL del servidor
db = "Donsson_produccion" # nombre de la base de datos


# --- Autenticación ---
common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")

# --- Fechas ---
from datetime import date, timedelta

# --- Fechas automáticas ---
hoy = date.today()
fecha_fin = hoy.strftime("%Y-%m-%d")
fecha_inicio = (hoy - timedelta(weeks=12)).strftime("%Y-%m-%d")
# --- 1) Buscar facturas válidas ---

invoice_domain = [
    ("date_invoice", ">=", fecha_inicio),
    ("date_invoice", "<=", fecha_fin),
    ("type", "=", "out_invoice"),   # solo ventas
    ("state", "in", ["open", "paid"])
]

invoice_ids = models.execute_kw(
    db, uid, password,
    "account.invoice", "search",
    [invoice_domain]
)
print(f"Facturas encontradas: {len(invoice_ids)}")

# --- 2) Descargar las líneas de esas facturas ---
fields = ["product_id", "quantity", "price_subtotal", "invoice_id","create_date","company_id","store"] #company id no va siempre muestra donsson

records = []
limit = 20000
offset = 0

while True:
    result = models.execute_kw(
        db, uid, password,
        "account.invoice.line", "search_read",
        [[("invoice_id", "in", invoice_ids)]],
        {"fields": fields, "limit": limit, "offset": offset}
    )
    if not result:
        break
    records.extend(result)
    offset += limit
    print(f"Descargados {len(records)} registros...")

# --- 3) Pasar a DataFrame ---
df = pd.DataFrame(records).fillna(0)

# --- 3) Pasar a DataFrame ---
df = pd.DataFrame(records).fillna(0)

# Separar product_id
df["product_id_num"] = df["product_id"].apply(
    lambda x: x[0] if isinstance(x, (list, tuple)) else None
)
df["product_name"] = df["product_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) else str(x)
)

# Separar invoice_id
df["invoice_id_num"] = df["invoice_id"].apply(
    lambda x: x[0] if isinstance(x, (list, tuple)) else None
)
df["invoice_name"] = df["invoice_id"].apply(
    lambda x: x[1] if isinstance(x, (list, tuple)) else str(x)
)

df["company_id"] = df["company_id"].astype(str)

# Convertir fecha a datetime
df["date_invoice"] = pd.to_datetime(df["create_date"], errors="coerce")

# Eliminar las columnas originales problemáticas
df = df.drop(columns=["product_id", "invoice_id","create_date"])


df.to_parquet("historico3mesesEMA.parquet", index=False)

print(f"Total de líneas descargadas: {len(df)}")

Facturas encontradas: 12172
Descargados 20000 registros...
Descargados 40000 registros...
Descargados 46973 registros...
Total de líneas descargadas: 46973


# COSTO DEL PRODUCTO

In [ ]:
import xmlrpc.client
import pandas as pd


# ===============================
# Conexión con Odoo
# ===============================
username = "juan.cano@donsson.com"   # tu usuario
password = "1000285668"              # tu contraseña
url = "https://donsson.com"          # URL del servidor
db = "Donsson_produccion"            # nombre de la base de datos

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")


#COSTO DE LOS PRODUCTOS

campos = models.execute_kw(
    db , uid , password,
    "cs.analisis.costo","fields_get",
    [],

)

df_campos = pd.DataFrame(campos)
#df_campos.info()


productos_costo = models.execute_kw(
    db , uid , password,
    "cs.analisis.costo","search_read",
    [[]],
    {"fields":["producto_costo_unitario","producto_id"]}


)

# Convertir en df
df_productos_costo = pd.DataFrame(productos_costo)


# Separar lista anidada
df_productos_costo[["product_id_num","product_name"]] = (df_productos_costo["producto_id"].apply(pd.Series))

#Agregar columna estandarizada
df_productos_costo["product_name"] = df_productos_costo["product_name"].str.strip()

#DF normalizado
df_productos_costo = df_productos_costo[["product_name","producto_costo_unitario"]]

df_productos_costo.to_parquet("costo_productos.parquet")


# VENTAS PERDIDAS

In [ ]:
import xmlrpc.client 
from datetime import date
import pandas as pd

# ===============================
# Conexión con Odoo
# ===============================
username = "juan.cano@donsson.com"   # tu usuario
password = "1000285668"              # tu contraseña
url = "https://donsson.com"          # URL del servidor
db = "Donsson_produccion"            # nombre de la base de datos

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")

# ===============================
# Leer ventas perdidas desde 2025
# ===============================
ventas_perdidas_ids = models.execute_kw(
    db, uid, password,
    "ventas.perdidas", "search",
    [[["fecha", ">=", "2024-01-01"]]]
)

ventas_perdidas = models.execute_kw(
    db, uid, password,
    "ventas.perdidas", "read",
    [ventas_perdidas_ids],
    {"fields": ["origen", "store_id", "warehouse_id", "cliente_id",
                "fecha", "product_ref", "product_id",
                "cantidad", "cantidad_existencia"]}
)

df = pd.DataFrame(ventas_perdidas)

# ===============================
# Normalizar columnas con tuplas (many2one)
# ===============================
def extract_id(val):
    if isinstance(val, (list, tuple)) and len(val) > 0:
        return val[0]
    return None

def extract_name(val):
    if isinstance(val, (list, tuple)) and len(val) > 1:
        return val[1]
    return str(val)

df["product_id_num"] = df["product_id"].apply(extract_id)
df["product_name"]   = df["product_id"].apply(extract_name)

df["cliente_id_num"] = df["cliente_id"].apply(extract_id)
df["cliente_name"]   = df["cliente_id"].apply(extract_name)

df["store_id_num"] = df["store_id"].apply(extract_id)
df["store_name"]   = df["store_id"].apply(extract_name)

df["warehouse_id_num"] = df["warehouse_id"].apply(extract_id)
df["warehouse_name"]   = df["warehouse_id"].apply(extract_name)

# ===============================
# Forzar tipos seguros
# ===============================
for col in df.select_dtypes(include="object").columns:
    df[col] = df[col].astype("string")

# ===============================
# Exportar a Parquet
# ===============================
try:
    df.to_parquet("ventas_perdidas_2025.parquet", index=False, engine="pyarrow", compression="snappy")
    print("✅ Exportación exitosa con pyarrow")
except Exception as e:
    print("⚠️ pyarrow falló, probando con fastparquet:", e)
    df.to_parquet("ventas_perdidas_2025.parquet", index=False, engine="fastparquet", compression="snappy")
    print("✅ Exportación exitosa con fastparquet")



✅ Exportación exitosa con pyarrow


In [ ]:
import xmlrpc.client
import pandas as pd

# ===============================
# Conexión con Odoo
# ===============================
username = "juan.cano@donsson.com"   # tu usuario
password = "1000285668"              # tu contraseña
url = "https://donsson.com"          # URL del servidor
db = "Donsson_produccion"            # nombre de la base de datos

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")

# ===============================
# Ver campos disponibles en hr.payslip.line
# ===============================
fields = models.execute_kw(
    db, uid, password,
    "hr.payslip.line", "fields_get",
    []
)

for field, info in fields.items():
    print(f"{field:25} | {info.get('type')} | {info.get('string')}")


code                      | char | Code
create_date               | datetime | Created on
novedad_id                | many2many | Novedeades
sequence                  | integer | Sequence
write_uid                 | many2one | Last Updated by
rate                      | float | Rate (%)
worked_days_line          | many2one | Novedad
appears_on_payslip        | boolean | Appears on Payslip
condition_range           | char | Range Based on
total                     | float | Total
id                        | integer | Id
amount_fix                | float | Fixed Amount
create_uid                | many2one | Created by
employee_id               | many2one | Employee
display_name              | char | Display Name
contract_id               | many2one | Contract
__last_update             | datetime | Last Modified on
company_id                | many2one | Company
note                      | text | Description
amount_percentage         | float | Percentage (%)
condition_range_min       | flo

In [2]:
import xmlrpc.client
import pandas as pd

# ===============================
# Conexión con Odoo
# ===============================
username = "juan.cano@donsson.com"
password = "1000285668"
url = "https://donsson.com"
db = "Donsson_produccion"

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")

# 1) Traer nóminas (slips) solo de 2025
# ========================================
slips = models.execute_kw(
    db, uid, password,
    "hr.payslip", "search_read",
    [[["date_from", ">=", "2024-01-01"], ["date_to", "<=", "2025-12-31"]]],
    {"fields": ["id", "employee_id", "name", "date_from", "date_to"]}
)
df_slips = pd.DataFrame(slips)


# ========================================
# 2) Traer líneas de nómina (solo de los slips obtenidos)
# ========================================
slip_ids = df_slips["id"].tolist()
lines = models.execute_kw(
    db, uid, password,
    "hr.payslip.line", "search_read",
    [[["slip_id", "in", slip_ids]]],
    {"fields": ["id", "slip_id", "employee_id", "salary_rule_id", "total"]}
)
df_lines = pd.DataFrame(lines)

# ===============================
# 3) Expandir campos Many2one (id, name)
# ===============================
def extract_name(val):
    if isinstance(val, list) and len(val) == 2:
        return val[1]
    return None

def extract_id(val):
    if isinstance(val, list) and len(val) == 2:
        return val[0]
    return None

df_lines["employee_name"] = df_lines["employee_id"].apply(extract_name)
df_lines["slip_name"]     = df_lines["slip_id"].apply(extract_name)
df_lines["rule_name"]     = df_lines["salary_rule_id"].apply(extract_name)

# 🔑 Extraer ID de slip_id para poder hacer el merge
df_lines["slip_id_only"] = df_lines["slip_id"].apply(extract_id)

# ===============================
# 4) Unir líneas con fechas de slip
# ===============================
df_slips["employee_name"] = df_slips["employee_id"].apply(extract_name)

df = pd.merge(
    df_lines,
    df_slips[["id", "employee_name", "date_from", "date_to"]],
    left_on="slip_id_only",
    right_on="id",
    how="left",
    suffixes=("", "_slip")
)

# ===============================
# 5) Filtrar reglas relevantes
# ===============================
df_filtro = df[df["rule_name"].isin(
    ["Salario Básico", "Salario Mínimo", "Salario Neto"]
)].copy()

# ===============================
# 6) Agregar columna período (Año-Mes)
# ===============================
df_filtro["periodo"] = pd.to_datetime(df_filtro["date_from"]).dt.to_period("M")

# ===============================
# 7) Pivotear para resumir salarios
# ===============================
tabla = df_filtro.pivot_table(
    index=["employee_name", "periodo"],
    columns="rule_name",
    values="total",
    aggfunc="sum",
    fill_value=0
).reset_index()

# ===============================
# 8) Renombrar columnas
# ===============================
tabla = tabla.rename(columns={
    "Salario Básico": "salario_basico",
    "Salario Mínimo": "salario_minimo",
    "Salario Neto": "salario_neto"
})

# ===============================
# 9) Calcular salario base = máximo entre básico y mínimo
# ===============================
tabla["salario_base"] = tabla[["salario_basico", "salario_minimo"]].max(axis=1)

# ===============================
# 10) Tabla final
# ===============================
resultado = tabla[["employee_name", "periodo", "salario_base", "salario_neto"]]

print("✅ Resumen mensual de salarios")


✅ Resumen mensual de salarios


In [24]:
r =resultado.sort_values(["salario_neto"],ascending=False)
r = r[r["periodo"]=="2025-07"]
r.head(18)

rule_name,employee_name,periodo,salario_base,salario_neto
886,GERMAN RICARDO BETANCOURT GUTIERREZ,2025-07,2847000.0,65057400.0
1302,JUAN IGNACIO NIETO NEIVA,2025-07,6960000.0,15046612.0
1209,JUAN CARLOS BETANCOURT GUTIERREZ,2025-07,8800000.0,9343639.0
1973,SIMON BETANCOURT DEL PINO,2025-07,10700000.0,9273600.0
18,ALBA LUCIA CASASBUENAS LOPEZ,2025-07,9613333.0,8935600.0
1993,SIOMARA SORETH HERNANDEZ OCHOA,2025-07,9613333.0,8935600.0
266,BRAYAN NAIR MUÑOZ SALAS,2025-07,8100000.0,8684867.0
977,JAIME ALBERTO CAÑON RAMIREZ,2025-07,2847000.0,7299075.0
93,ANDREA KATERYNE ALVARADO GONZALEZ,2025-07,3500000.0,7212867.0
1079,JESUS ADONAY GONZALEZ FLOREZ,2025-07,2847000.0,7030985.0
